In [3]:
import pandas as pd
import os

new_df = pd.read_csv(os.path.join("..","..","data","TA_User_Reviws_Korea_all_v2_new_df.csv"))
new_df = new_df.drop(['location_name'], axis=1)
new_df.head()  

,locationId,rated_count,average_photonum,average_rating,userID,users_mean_rating,user_reviewcount,rating,placeType,createdDate,islocal,photonum
0,299152,13,2.846154,3.692308,4842,3.677083,96,5,ACCOMMODATION,20171009,1,12
1,299152,13,2.846154,3.692308,9212,3.791667,240,3,ACCOMMODATION,20180306,1,0
2,299152,13,2.846154,3.692308,14262,4.500000,2,4,ACCOMMODATION,20121220,0,1
3,299152,13,2.846154,3.692308,20235,3.000000,4,1,ACCOMMODATION,20161127,0,4
4,299152,13,2.846154,3.692308,21432,4.200000,5,4,ACCOMMODATION,20120416,0,0


### 로컬 / 글로벌 데이터 분리

In [4]:
# 로컬 / 글로벌 데이터 분리
local_df = new_df.loc[new_df['islocal']==1]
global_df = new_df.loc[new_df['islocal']==0]
print('local_df shape:',local_df.shape, 'global_df shape :',global_df.shape)

local_df shape: (43294, 12) global_df shape : (94196, 12)


## 1. local_modeling

In [5]:
def get_data(column, df) :
    
    vocab = {}
    num = 0

    for i in df[column]: # np.hstack([train[column], test[column]]): 
        if vocab.get(i) != None:
            continue

        vocab[i] = num
        num += 1

    encoded = [vocab[i] for i in df[column]]
    # encoded_d = [vocab[i] for i in test[column]]
    
    return encoded, num, vocab

In [6]:
encoded_userID, num_userID, vocab_userID = get_data('userID', local_df) 

encoded_users_mean_rating, num_users_mean_rating, vocab_users_mean_rating = get_data('users_mean_rating', local_df) 

encoded_user_reviewcount,  num_user_reviewcount, vocab_user_reviewcount = get_data('user_reviewcount', local_df) 

encoded_locationId, num_locationId, vocab_locationId = get_data('locationId', local_df) 
encoded_placeType,  num_placeType, vocab_placeType = get_data('placeType', local_df) 

encoded_createdDate,  num_createdDate, vocab_createdDate = get_data('createdDate', local_df) 
encoded_photonum,  num_photonum, vocab_photonum = get_data('photonum', local_df) 

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

from torchfm.layer import FeaturesEmbedding, FeaturesLinear, AttentionalFactorizationMachine

output_dim = 8 

In [ ]:
def FeaturesLinear(field_dims, output_dim):
    fc = torch.nn.Embedding(sum(field_dims), output_dim)
    bias = torch.nn.Parameter(torch.zeros((output_dim,)))
    offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)

    x = x + x.new_tensor(self.offsets).unsqueeze(0)
    return torch.sum(self.fc(x), dim=1) + self.bias


In [ ]:
def FeaturesEmbedding():

    embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
    offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
    torch.nn.init.xavier_uniform_(embedding.weight.data)

    x = x + x.new_tensor(offsets).unsqueeze(0)
    return embedding(x)


In [ ]:
class AttentionalFactorizationMachineModel(torch.nn.Module):

    def __init__(self, field_dims, embed_dim, attn_size, dropouts):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.linear = FeaturesLinear(field_dims)
        self.afm = AttentionalFactorizationMachine(embed_dim, attn_size, dropouts)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = self.linear(x) + self.afm(self.embedding(x))
        return torch.sigmoid(x.squeeze(1))

In [ ]:
num_fields = len(field_dims)
embedding = FeaturesEmbedding(field_dims, embed_dim)
linear = FeaturesLinear(field_dims)
afm = AttentionalFactorizationMachine(embed_dim, attn_size, dropouts)



In [ ]:
x = linear(x) + afm(embedding(x))
return torch.sigmoid(x.squeeze(1))